In [ ]:
!pip install scikit-image mrcnn

  Using cached skimage-0.0.tar.gz (757 bytes)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [3 lines of output]
      
      *** Please install the `scikit-image` package (instead of `skimage`) ***
      
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os
import sys
import random
import math
import datetime
import skimage.draw
import numpy as np
import matplotlib.pyplot as plt

# Root directory of the project
ROOT_DIR = os.path.abspath(".")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library

from mrcnn import utils
from mrcnn import visualize
from mrcnn import model as modellib

# Import COCO config (adjust if your custom config is different)
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco_car_poses  # Replace with your custom config module if needed

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Download weights if not present
model_path = "mask_rcnn_coco.h5"
if not os.path.exists(model_path):
    utils.download_trained_weights(model_path)

# Configuration for training on the dataset
config_train = coco_car_poses.TrainConfig()  # Adjust to your config class

# Dataset (assume dataset_train and dataset_val are defined in your custom module)
# Load dataset
# dataset_train = coco_car_poses.CocoDataset()  # Example; adjust as needed
# dataset_train.load_coco(...)  # Load your data
# dataset_train.prepare()

# Create model in training mode
model = modellib.MaskRCNN(mode="training", config=config_train, model_dir=MODEL_DIR)

# Augmentation (example from your code snippet)
# augmentation = imgaug.augmenters.Fliplr(0.5)

# Load weights
print("Loading weights", model_path)
model.load_weights(model_path, by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])


ModuleNotFoundError: No module named 'skimage'

In [ ]:
# Training - Stage 1
print("Training network heads")
model.train(dataset_train, dataset_train,
            learning_rate=config_train.LEARNING_RATE,
            epochs=10,
            layers='heads')

# Training - Stage 2
# Finetune layers from ResNet stage 4 and up
print("Fine tune Resnet stage 4 and up")
model.train(dataset_train, dataset_train,
            learning_rate=config_train.LEARNING_RATE,
            epochs=50,
            layers='4+')

# Training - Stage 3
# Fine tune all layers
print("Fine tune all layers")
model.train(dataset_train, dataset_train,
            learning_rate=config_train.LEARNING_RATE / 10,
            epochs=100,
            layers='all')


## Evalution by class

In [ ]:
inference_config = coco_car_poses.InferenceConfig()

model_inference = modellib.MaskRCNN(mode="inference", config=inference_config,
                          model_dir=MODEL_DIR)

In [ ]:
#coco_car_poses.evaluate_coco(model_inference, dataset_val, coco_train, "bbox")

In [ ]:
#coco_car_poses.evaluate_coco(model_inference, dataset_val, car_poses_coco, "segm")